In [1]:
import minsearch
import openai
import json

In [2]:
with open("documents.json", "rb") as f_in:
    docs_raw = json.load(f_in)

In [3]:
documents = []

for course in docs_raw:
    for doc in course["documents"]:
        doc['course'] = course["course"]
        documents.append(doc)

In [4]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [5]:
index = minsearch.Index(
    text_fields = ["question", "text", "section"],
    keyword_fields = ["course"]
)

In [6]:
q = "can i still enroll in the course after it has started"

In [7]:
index.fit(documents)

In [8]:
from openai import OpenAI

In [9]:
client = OpenAI()

In [10]:
response = client.chat.completions.create(
    model = "gpt-4o",
    messages=[{"role": "user", "content": q}]
)
response.choices[0].message.content

"Whether you can still enroll in a course after it has already started depends on several factors, including the policies of the institution offering the course, the specific course itself, and the point in the term at which you are attempting to enroll.\n\nHere are some steps you can take to find out:\n\n1. **Check the Institution's Policies**: Look at the academic policies on the institution's website. There may be specific guidelines for late enrollment or add/drop deadlines.\n\n2. **Contact the Instructor**: Sometimes, instructors have the discretion to allow students to join a course late. You can email the instructor to explain your situation and ask if late enrollment is possible.\n\n3. **Contact the Registrar or Academic Advisor**: University registrars or academic advisors can provide information on the policies and possible exceptions for enrolling in a course after it has started.\n\n4. **Assess Catch-Up Requirements**: Determine how much material you would need to catch up 

In [11]:
def search(query):
    boost = {"question": 3.0, "section": 0.5}

    results = index.search(
        filter_dict={"course" : "data-engineering-zoomcamp"},
        query = query,
        boost_dict = boost,
        num_results = 5
    )
    return results

In [12]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT: 
    {context}
    """.strip()
    
    context = ""
        
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
        
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [13]:
def llm(prompt):
    response = client.chat.completions.create(
    model = "gpt-4o",
    messages=[{"role": "user", "content": prompt}]
)
    return response.choices[0].message.content

In [14]:
query = "how do i build kafka"

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [15]:
rag("how do i get the certificate")

'To obtain a certificate for completing the course, you need to finish the course with a “live” cohort. Certificates are not awarded for the self-paced mode because you need to peer-review capstone projects after submitting your own project. Peer reviews can only be conducted while the course is actively running.'

In [16]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [17]:
from elasticsearch import Elasticsearch

In [18]:
es_client = Elasticsearch("http://localhost:9200")

In [19]:
es_client.info()

ObjectApiResponse({'name': 'd868578a64a0', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'FwfkZ7uQS_OsPX-eNl2QiA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [20]:
query = "I just disovered the course. Can I still join it?"

In [21]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}


index_name = "course-questions"
es_client.indices.create(index= index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [22]:
from tqdm.auto import tqdm

In [23]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [24]:
query = "I just disovered the course. Can I still join it?"

In [25]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    result_docs = []
    for hit in response["hits"]["hits"]:
        result_docs.append(hit['_source'])
    return result_docs

In [26]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [27]:
rag(query)

"Yes, you can still join the course even if you discovered it after the start date. You are eligible to submit the homeworks. However, be aware that there will be deadlines for turning in the final projects, so it's best not to leave everything for the last minute."

In [28]:
def homework(query):
    homework_body = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=homework_body)
    result_docs = []
    for hit in response["hits"]["hits"]:
        result_docs.append(hit['_score'])
    return result_docs

In [29]:
homework("How do I execute a command in a running docker container")

[75.54128, 43.922554, 38.684105, 38.33403, 35.94081]

In [30]:
def homework2(query):
    homework_body = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=homework_body)
    result_docs = []
    for hit in response["hits"]["hits"]:
        result_docs.append(hit['_source'])
    return result_docs

In [31]:
homework2('How do I execute a command in a running docker container?')

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [32]:
def build_prompt(query, search_results):
    context_template = """
    Q: {question}
    A: {text}
    """.strip()

    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT: 
    {context}
    """.strip()
    
    combined_context = ""
    for doc in search_results:
        context_template = f"Q: {doc['question']}\nA: {doc['text']}\n\n"
        combined_context += context_template
        
    prompt = prompt_template.format(question=query, context=combined_context).strip()
    return prompt

In [40]:
def build_prompt(query, search_results):
    context_template = """
Q: {question}
A: {text}
""".strip()

    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    combined_context = ""
    for doc in search_results:
        context_entry = context_template.format(question=doc['question'], text=doc['text'])
        combined_context += context_entry + "\n\n"

    # Remove the trailing double newline from the combined_context
    combined_context = combined_context.strip()

    prompt = prompt_template.format(question=query, context=combined_context).strip()
    return prompt

In [43]:
query = "How do I execute a command in a running docker container?"

def rag(query):
    search_results = homework2(query)
    prompt = build_prompt(query, search_results)
    return prompt

In [44]:
len(rag(query))

2714

In [45]:
prompt = rag(query)

In [46]:
import tiktoken

In [47]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [48]:
encoded_prompt = encoding.encode(prompt)
num_tokens = len(encoded_prompt)

In [49]:
num_tokens

621